# NFT - Etherscan - Verificar Quantidade de Transações de NFTs e gerar arquivo de saida com os Token Address  que possuem mais de 1500 transações

## Packages and libraries

In [1]:
pip install db-dtypes pandas tensorflow scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
# importar pacotes
import os
import requests
import numpy as np
import glob
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.svm import OneClassSVM
from sklearn.metrics import silhouette_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.cluster import DBSCAN

from tensorflow import keras
from tensorflow.keras import layers, models

In [3]:
# Chave Privada API Etherscan
api_key = "5YV6KJ19N4HIIT36FYWK894RIJUA1JXW3V"

In [26]:
def processar_arquivo_csv(caminho_arquivo_csv):
    # Ler o arquivo CSV
    df = pd.read_csv(caminho_arquivo_csv, sep=',')

    # Obter a quantidade de registros lidos
    total_registros_lidos = df.shape[0]
    print(f"Total de registros lidos: {total_registros_lidos}")

    # Conjunto para armazenar os NFTs IDs já processados
    nfts_processados = set()

    # Lista para armazenar as informações das NFTs
    lista_nfts1500 = []

    # Contador para o número de NFTs com mais de 1500 transações
    contador_nfts1500 = 0

    # Para cada linha do DataFrame
    for index, row in df.iterrows():
        nft_id = row['NFTToken_ID']

        # Verificar se o NFT já foi processado
        if nft_id in nfts_processados:
            continue

        nft_blocktime = row['DateTime_UTC']
        nft_name = row['NFTName']

        print(f"Processando NFT: {nft_id} - {nft_name} - {nft_blocktime}")

        # Chamar a função para verificar a quantidade de transações de NFT
        num_transactions = verificar_transacoes_nft(nft_id)

        # Se num_transactions for None, tratá-lo como 0
        qtde_transactions = int(num_transactions) if num_transactions is not None else 0

        if qtde_transactions > 1500:
            print(f"NFT selecionada para arquivo de saída. \n")

            # Adicionar as informações da NFT em um dicionário à lista
            lista_nfts1500.append({
                'nft_id': nft_id,
                'nft_name': nft_name,
                'nft_blocktime': nft_blocktime
            })

            # Incrementar o contador
            contador_nfts1500 += 1

        # Adicionar o NFT ID ao conjunto de processados
        nfts_processados.add(nft_id)

    return lista_nfts1500, contador_nfts1500, total_registros_lidos

In [27]:
# Função para verificar o número de transações de uma NFT
def verificar_transacoes_nft(nft_hash):
    
    # URL da API do Etherscan para obter o histórico de transações de um endereço
    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={nft_hash}&apikey={api_key}"
    try:
        # Solicitação à API do Etherscan
        response = requests.get(url)
        data = response.json()

        # Verificação se a solicitação foi bem-sucedida
        if data["status"] == "1":
            # Obter o número de transações
            num_transactions = len(data["result"])
            
            # Verificar se o número de transações é maior que 1500
            if num_transactions > 1500:
                print(f"A NFT possui mais de 1500 transações. Foram {num_transactions} totalizadas.")
            else:
                print(f"A NFT possui {num_transactions} transações, que não atendem ao critério de mais de 1500 \n")
            return num_transactions
        else:
            print("Ocorreu um erro na solicitação à API do Etherscan.")
    except Exception as e:
        print(f"Erro ao fazer a solicitação à API do Etherscan: {str(e)}")

In [33]:
def exportar_para_csv(lista_nfts, caminho_arquivo_saida):
    # Criar um DataFrame a partir da lista de dicionários
    df = pd.DataFrame(lista_nfts)

    # Exportar o DataFrame para um arquivo CSV
    df.to_csv(caminho_arquivo_saida, index=False, sep=',')

In [29]:
lista_resultados = []

# Caminho do arquivo CSV de entrada
caminho_arquivo_csv = 'Z:/Python/Data/Coleta_NFT/Abril/bq-results-20230401-20230430_Mint.csv'

# Chamar a função principal para processar o arquivo CSV e receber os valores retornados
lista_resultados, total_nfts1500, total_registros_lidos = processar_arquivo_csv(caminho_arquivo_csv)

# Imprimir o total de NFTs com mais de 1500 transações e o total de registros lidos
print(f"Total de NFTs com mais de 1500 transações: {total_nfts1500}")
print(f"Total de registros lidos do arquivo: {total_registros_lidos}")

Total de registros lidos: 10215
Processando NFT: 0x6f7fe413e871b80c5a12c180f5ab3964f82d5cdc - Omniguards - 2023-04-01 00:05:47 UTC
A NFT possui mais de 1500 transações.
NFT selecionada para arquivo de saída. 

Processando NFT: 0xb932a70a57673d89f4acffbe830e8ed7f75fb9e0 - SuperRare - 2023-04-01 00:23:59 UTC
A NFT possui mais de 1500 transações.
NFT selecionada para arquivo de saída. 

Processando NFT: 0x06012c8cf97bead5deae237070f9587f8e7a266d - CryptoKitties - 2023-04-01 00:34:35 UTC
A NFT possui mais de 1500 transações.
NFT selecionada para arquivo de saída. 

Processando NFT: 0x696418f16804228d86266e12db5c5178e81a37c0 - Casa De Luna - 2023-04-01 00:59:11 UTC
A NFT possui mais de 1500 transações.
NFT selecionada para arquivo de saída. 

Processando NFT: 0xa3d58c4e56fedcae3a7c43a725aee9a71f0ece4e - Metronome - 2023-04-01 01:07:23 UTC
A NFT possui mais de 1500 transações.
NFT selecionada para arquivo de saída. 

Processando NFT: 0xddb3422497e61e13543bea06989c0789117555c5 - COTI Token - 

A NFT possui 320 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0xd71ecff9342a5ced620049e616c5035f1db98620 - Synth sEUR - 2023-04-02 22:33:35 UTC
A NFT possui mais de 1500 transações.
NFT selecionada para arquivo de saída. 

Processando NFT: 0xec42d7ca35dae8eab130f542e71806d21db268b4 - DeY00ts (33.3%) - 2023-04-02 23:48:23 UTC
A NFT possui mais de 1500 transações.
NFT selecionada para arquivo de saída. 

Processando NFT: 0xb5385d48318531cf2365b74b61b44fc352a9a82d - Humble Beings Collective - 2023-04-03 00:01:59 UTC
A NFT possui 134 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0xa56a4f2b9807311ac401c6afba695d3b0c31079d - MonsterBlocks - 2023-04-03 04:36:35 UTC
A NFT possui mais de 1500 transações.
NFT selecionada para arquivo de saída. 

Processando NFT: 0x4f718f17ba247ad9e8b6f63de0ed5f3868bc90f5 - NFA'S ToonyMoney by ToonyMania  - 2023-04-03 08:10:35 UTC
A NFT possui 330 transações, que não atendem ao critério de mais de 1500 



A NFT possui 346 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x0778831ed1abe4051f82d9a4b637b06b37cf487b - Pikachu Games - 2023-04-06 21:34:11 UTC
A NFT possui 4 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x8125afd067094cd573255f82795339b9fe2a40ab - Wrapped Ether - 2023-04-06 23:17:23 UTC
A NFT possui 16 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x3dd0b89f227dddf9112094de93afbd6d50aa9d00 - The Eye - 2023-04-07 00:20:47 UTC
A NFT possui 13 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0xa7cb7f55ece2d1df3379f303f33dd3e1ee593b8f - Wealth Builder Token - 2023-04-07 02:06:35 UTC
A NFT possui 584 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x9cb02f5daf5d9a807b3d9732d2e9da4f5ddc33ef - Flowers & Friends - 2023-04-07 02:28:59 UTC
A NFT possui 6 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0xc72633f995e98ac3bb8a89

A NFT possui mais de 1500 transações.
NFT selecionada para arquivo de saída. 

Processando NFT: 0x8cf95e6ea4b1a38c7bc59554564990a9bb37cf55 - MIYAGIDOJO - 2023-04-10 08:40:47 UTC
A NFT possui 21 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x6b6cdd6ed60c9faad222b0d9645713b5cf0d3b0f - SpinnakerX Token - 2023-04-10 11:42:47 UTC
A NFT possui 4 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0xb7ec7bbd2d2193b47027247fc666fb342d23c4b5 - Mirage Gallery Curated - 2023-04-10 16:30:47 UTC
A NFT possui mais de 1500 transações.
NFT selecionada para arquivo de saída. 

Processando NFT: 0x1ef19217b34c063b8b24aa26a87769dac8f79ec6 - Circle - 2023-04-10 18:47:35 UTC
A NFT possui 99 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0xb7d9968f8b256bdf08bf8062fc26d5e3eb912337 - Gwaiverse - 2023-04-10 22:50:47 UTC
A NFT possui 47 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0xcb12e617c17598eda4eb

A NFT possui mais de 1500 transações.
NFT selecionada para arquivo de saída. 

Processando NFT: 0x16bc47d5029f61d2983a88ee9255bf883e8920e4 - IndigoIndiaCoin - 2023-04-16 16:40:47 UTC
A NFT possui 1 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x7f737b66921d18b008dc9c3a9b933a3bf559f1bc - Interface Token - 2023-04-16 17:08:59 UTC
A NFT possui 181 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0xb83e39f96593e50aed4640731b1a17ab71289c1a - Granny's Nifty Aipes - 2023-04-16 19:21:59 UTC
A NFT possui 21 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x4f9766bc88eb0d42056d48e2ddc4d1ff2f06018c - b₿ - 2023-04-17 02:45:59 UTC
A NFT possui 301 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x1ce53727f7116d6e9b62f60af88a825690ff9187 - Ethereum - 2023-04-17 09:59:59 UTC
A NFT possui 45 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0xc865e63345583e213d799fc8d6

A NFT possui 29 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x00d3cf9ccc40debd2e2920256facb3dbf76f9dd8 - RickRoll - 2023-04-21 15:07:11 UTC
A NFT possui 122 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x5ed113254705ebe07605d7921eb1d06b23e1a3ac - GENESIS TOKEN - 2023-04-21 15:07:47 UTC
A NFT possui 2 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x95f81442f271ea94f1744be72b3dd44c208665aa - VORTEX TOKEN - 2023-04-21 15:15:35 UTC
A NFT possui 13 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0xba745513acebcbb977497c569d4f7d340f2a936b - Mainstream For The Underground - 2023-04-21 15:31:35 UTC
A NFT possui 523 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x8fde244c1a8be4b30f9872e960a29c3b2cbf1410 - TITAN - 2023-04-21 17:03:35 UTC
A NFT possui 8 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0xee3926689896ec462aed8bc7

A NFT possui 89 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x6858e512e829ed633140dcc19746781c8e5b1049 - PPPALS - 2023-04-26 09:38:59 UTC
A NFT possui 120 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x28b63f4d5c37dd1416b6400ae9e1c04ee449ffbd - Adventure Time - 2023-04-26 11:24:59 UTC
A NFT possui 13 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x6be2871bf1b7fbd6d4fc13c1de7105f82b758f6d - Harold - 2023-04-26 15:13:11 UTC
A NFT possui 8 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0xf2ab2f87088f35ff79af516bba701470f407b396 - AVATAR⚊ - 2023-04-26 15:20:59 UTC
A NFT possui 20 transações, que não atendem ao critério de mais de 1500 

Processando NFT: 0x9081ceb359b6f42640651e11f5bfb4d4b84aa66f - Partial Basic - 2023-04-26 15:53:23 UTC
A NFT possui mais de 1500 transações.
NFT selecionada para arquivo de saída. 

Processando NFT: 0xd3678191f3177032e06e6c1b553ec9fa841b6b48 

In [35]:
caminho_saida_csv = 'Z:/Python/Data/Coleta_NFT/Abril/Saida_NFTs1500_Apr_2023.csv'

# Chamando a função para exportar os dados
exportar_para_csv(lista_resultados, caminho_saida_csv)